# 1. 获取数据

In [1]:
import pandas as pd

data = pd.read_csv('news_chinese.csv')
data = data.dropna()

# 2. 获得所有表示“说”的意思的单词

In [2]:
# -*- coding: utf-8 -*-
import os
LTP_DATA_DIR = '.\\ltp_data_v3.4.0\\'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

def cut_words(text):
    words = segmentor.segment(text)  # 分词
    return words
#     segmentor.release()  # 释放模型

In [3]:
from pyltp import SentenceSplitter

def split_sentence(text):
    '''
    split text into single sentence without '\n'
    '''
    sentences = []
    try:
        sentence_list = text.split('\\n')
        for sentence in sentence_list:
            splited_sentence = SentenceSplitter.split(sentence)  # 分句
            sentences.extend(list(splited_sentence))
    except Exception:
        print('split_sentence exception:' + str(text))
        print(Exception)
    return sentences

In [4]:
def preprocess_text(text):
    """
    preprocess text, drop number, blank, stopwords
    return segments list
    """
#     stopwords=pd.read_csv('.\\stopwords.txt',index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
#     stopwords=stopwords['stopword'].values
    if len(text) == 1:
        return ''
    
    try:
        segs = list(cut_words(text))
#         segs = [v for v in segs if not str(v).isdigit()]#去数字
        segs = list(filter(lambda x:x.strip(), segs)) #去左右空格
#         segs = list(filter(lambda x:len(x)>1, segs))#长度为1的字符
#         segs = list(filter(lambda x:x not in stopwords, segs)) #去掉停用词
    except Exception:
        print('preprocess_text exception:' + str(text))
        print(Exception)
    return segs

In [5]:
'''
preprocess and write corpus to file for future use
'''
with open('corpus.txt', 'w', encoding='utf-8') as f:
    for graph in data['content'].values:
        sentences = split_sentence(graph)  # 分句
        for sentence in sentences:
            words = preprocess_text(sentence)
            if words == '':
                continue
            f.write(' '.join(words))
            f.write('\n')

In [6]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
corpus = LineSentence('corpus.txt')
'''
LineSentence(inp)：格式简单：一句话=一行; 单词已经过预处理并被空格分隔。
size：是每个词的向量维度； 
window：是词向量训练时的上下文扫描窗口大小，窗口为5就是考虑前5个词和后5个词； 
min-count：设置最低频率，默认是5，如果一个词语在文档中出现的次数小于5，那么就会丢弃； 
workers：是训练的进程数（需要更精准的解释，请指正），默认是当前运行机器的处理器核数。这些参数先记住就可以了。
sg ({0, 1}, optional) – 模型的训练算法: 1: skip-gram; 0: CBOW
alpha (float, optional) – 初始学习率
iter (int, optional) – 迭代次数，默认为5
'''
model = Word2Vec(sentences=corpus, size=100, window=5, min_count=1, sg=1)

model.save(".\\word2vec.model")

E:\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
E:\Anaconda3\envs\py36\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
model['说']

E:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.20866792,  0.7205171 , -1.0260113 , -0.06374749,  0.19556347,
       -0.10682994, -0.5851775 , -0.01481899,  0.3670453 ,  0.24029326,
        0.5949725 ,  0.28039786, -0.06872837, -0.33250424, -0.1844618 ,
       -0.5172169 ,  0.34398785, -0.58655286, -0.1243936 , -0.75491464,
        0.09888597,  0.40916127,  0.10244063,  0.7225233 ,  0.32497236,
        0.56063825,  0.8444974 , -0.3222012 ,  0.34923565,  0.27028653,
       -0.51589775, -0.5600994 ,  0.2264611 , -0.385016  , -0.4579577 ,
        0.08523443,  0.64386386, -0.10020302, -0.01174884, -0.51918715,
       -0.01156036, -0.22992441,  0.77415377,  0.05849807, -0.2401069 ,
       -0.00274309, -0.7530591 ,  0.07110906, -0.13691097,  0.1307326 ,
       -0.49360964, -0.21363465,  0.7133124 ,  0.357119  ,  0.47492936,
       -0.23782545,  1.0265316 , -0.13638072, -0.27368957, -0.03718648,
        0.7090336 , -0.76925814, -0.48834   ,  0.71585935,  0.19875243,
       -0.85786104,  0.28460357, -0.29654166, -0.14113064,  0.87

## 2.1 找相似词

Word Embedding Tutorial: word2vec using Gensim

https://www.guru99.com/word-embedding-word2vec.html

In [1]:
'''
load gensim word2vec model
'''
from gensim.models import Word2Vec
model = Word2Vec.load(".\\word2vec.model")

E:\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
E:\Anaconda3\envs\py36\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [2]:
vocab = list(model.wv.vocab)
len(vocab)

223587

In [4]:
similar_words = model.most_similar('说', topn=20)
print(similar_words)

E:\Soft\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('表示', 0.8741297125816345), ('告诉', 0.8271186351776123), ('指出', 0.788396954536438), ('认为', 0.7727850675582886), ('介绍', 0.7356007695198059), ('说道', 0.7067561745643616), ('看来', 0.6960127353668213), ('写道', 0.6913279891014099), ('话', 0.6870427131652832), ('强调', 0.6705794334411621), ('坦言', 0.6692733764648438), ('提到', 0.6580647230148315), ('称', 0.6515368223190308), ('透露', 0.6492000222206116), ('举例', 0.6212161779403687), ('形容', 0.611984372138977), ('证实', 0.6084652543067932), ('得知', 0.606695294380188), ('说法', 0.6029568910598755), ('问', 0.6021037101745605)]


In [5]:
similarity_two_words = model.similarity('说','讲')
print("Please provide the similarity between these two words:")
print(similarity_two_words)

Please provide the similarity between these two words:
0.3628039284301186


E:\Soft\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [6]:
similar = model.similar_by_word('说', topn=20)
print(similar)

[('表示', 0.8741297125816345), ('告诉', 0.8271186351776123), ('指出', 0.788396954536438), ('认为', 0.7727850675582886), ('介绍', 0.7356007695198059), ('说道', 0.7067561745643616), ('看来', 0.6960127353668213), ('写道', 0.6913279891014099), ('话', 0.6870427131652832), ('强调', 0.6705794334411621), ('坦言', 0.6692733764648438), ('提到', 0.6580647230148315), ('称', 0.6515368223190308), ('透露', 0.6492000222206116), ('举例', 0.6212161779403687), ('形容', 0.611984372138977), ('证实', 0.6084652543067932), ('得知', 0.606695294380188), ('说法', 0.6029568910598755), ('问', 0.6021037101745605)]


E:\Soft\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


In [4]:
def find_similar_words(words, max_size, model):
    similar_words = []
    seen = []
    unseen = ['说']
    while len(similar_words) < max_size and len(unseen) != 0:
        unseen_word = unseen.pop(0)
        if unseen_word in similar_words:
            continue
        similars = [w for w, p in model.most_similar(unseen_word, topn=20)]
        unseen.extend(similars)
        similar_words.append(unseen_word)
        seen.append(unseen_word)
    return similar_words
    

In [5]:
import jieba

words = []
with open('corpus.txt', 'r', encoding='utf-8') as rf:
    for line in rf.readlines():
        words.extend(list(jieba.cut(line)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\vick_\AppData\Local\Temp\jieba.cache
Loading model cost 1.100 seconds.
Prefix dict has been built succesfully.


In [9]:
len(words)

33172819

In [6]:
similar_words = find_similar_words(words, 20, model)
similar_words

E:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


['说',
 '表示',
 '告诉',
 '指出',
 '认为',
 '说道',
 '坦言',
 '介绍',
 '看来',
 '透露',
 '写道',
 '举例',
 '强调',
 '深有感触',
 '称',
 '话',
 '直言',
 '说法',
 '提到',
 '笑言']

# 3. 使用 NER，Dependency Parsing等对句子形式进行解析

In [6]:
similar_words = ['说',
 '表示',
 '告诉',
 '指出',
 '认为',
 '说道',
 '坦言',
 '介绍',
 '看来',
 '透露',
 '写道',
 '举例',
 '强调',
 '深有感触',
 '称',
 '话',
 '直言',
 '说法',
 '提到',
 '笑言']

In [7]:
# -*- coding: utf-8 -*-
import os
LTP_DATA_DIR = '.\\ltp_data_v3.4.0\\'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl_win.model')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型

from pyltp import NamedEntityRecognizer
recognizer = NamedEntityRecognizer() # 初始化实例
recognizer.load(ner_model_path)  # 加载模型

from pyltp import Parser
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

from pyltp import SementicRoleLabeller
labeller = SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型

In [8]:
def contain_keywords(keywords, words):
    '''
    return key word in words, return empty if not match
    '''
    for keyword in keywords:
        if keyword in words:
            return keyword
    return ''

In [9]:
maxsize = 100
count = 0

subs = []
says = []
points = []

with open('corpus.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        words = line.split(' ')
        keyword = contain_keywords(similar_words, words) # 句子是否包含“说”的同义词
        if keyword == '':
            continue
#         print('%d:%s' % (words.index(keyword), keyword))
        postags = postagger.postag(words)  # 词性标注
        netags = recognizer.recognize(words, postags)  # 命名实体识别
        arcs = parser.parse(words, postags)  # 句法分析
        
        # 打印词性标注，命名实体识别，句法分析结果(可注释)
#         print('序号' + '\t' + '词' + '\t' + '词性' + '\t' + '命名实体' + '\t' + '依存句法')
#         print('--------------------------------------------')
#         word_index = 0
#         for w,p,n,a in zip(words, postags, netags, ("%d:%s" % (arc.head, arc.relation) for arc in arcs)):
#             print(str(word_index) + '\t' + w + '\t' + p + '\t' + n + '\t' + a)
#             word_index += 1
        
        # arcs 使用依存句法分析的结果
        roles = labeller.label(words, postags, arcs)  # 语义角色标注
        
        # 打印依存句法分析结果(可注释)
#         for role in roles:
#             print(role.index, "".join(
#                 ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))
        
        for role in roles:
            if words.index(keyword) == role.index:
                for arg in role.arguments:
                    if arg.name == 'A0':
                        sub = ''.join(words[arg.range.start:arg.range.end+1])
                        point = ''.join(words[role.index+1:])
#                         print("观点主体: " + sub)
#                         print("观点内容: " + point)
                        subs.append(sub)
                        says.append(keyword)
                        points.append(point)
        
#         print('\n======================================================================\n')
        count += 1
        if count == maxsize:
            break

postagger.release()  # 释放模型
recognizer.release()  # 释放模型
parser.release()  # 释放模型
labeller.release()  # 释放模型

In [10]:
for s, v, p in zip(subs, says, points): 
    print(s + '\t' + v + '\t' + p)

来自广州的茶茶	说	。

她	告诉	父母这次出来是到德国看朋友，住宿和球票的花费都是平时做家教挣的，父母支援了机票钱，从香港到杜塞尔多夫往返只有三千多元人民币，让其他内地球迷非常羡慕。

熊老师	说	，这次有了德国的签证，以后去日本看比赛签证就容易多了。

井井	说	。

井井	说	。

茶茶	说	。

她	写道	。

哈佛大学招生部门	表示	要审核他们的入学情况。

哈佛大学发言人理查德·戴恩	说	，校方不会公开讨论申请入学者情况。

约60%的受访者	表示	，他们对马克龙的表现感到满意。

甘肃省交通运输厅	介绍	，此次集中开建35个重点交通建设项目，在进一步完善路网结构的同时，有助于促进甘肃经济运行趋稳向好、确保完成全年固定资产投资目标任务。

韩国国防部官员	说	，针对“萨德”在韩部署事宜，国防部方面已准备重新进行一场大规模的环境评估。

韩国国防部官员	透露	，青瓦台要求彻查“萨德”环境评估过程后，国防部已着手开展准备工作，将进行一场大规模的全面评估，预计可能耗时一年左右。

这名官员	透露	，目前，针对“萨德”用地的小规模环境评估正在进行中，预计将于本月底结束。

青瓦台方面	表示	，经调查发现，国防部在“萨德”环境评估过程中存在“猫腻”，试图“大事化小”，绕过相关法律规定。

青瓦台方面	表示	，总统文在寅获悉除目前已在韩部署的两台“萨德”系统移动发射架外，另有4台发射架已经秘密运抵韩国，感到“非常震惊”，要求彻查。

青瓦台公报首席秘书尹永灿	说	，调查已确认，瞒报事件主要责任人是国防部政策室长魏昇镐。

青瓦台方面	表示	，将就此事展开进一步调查，以查明国防部长官韩民求和前总统朴槿惠时期的国家安保室长金宽镇是否与瞒报事件有关联。

肖云峰	说	，除了水果玉米，他还种植了爆米花玉米等，收入也很可观。

肖云峰	认为	，大学生职业农民最大的优势在于对国家惠农政策的理解，以及对农业技术的掌控。

沈丘县农牧局局长童玉体	说	：“别人种玉米，他种水果玉米；

王晓康	说	，苗圃种植园经过整合会成为休闲农庄，不仅可以通过优质别样的苗木促进农业增收，也有利于农村生态，通过生态旅游创收。

王文一	说	。

周口市农业局科教科科长刘民乾	说	：“能够吸引众多大学生回归田野进行创业，是因为土地效益的巨大吸引力，而土地效益源于他们对市场的敏锐、对技术的把握以及科学的